# Getting labelled terms from Argilla, to improve explorer inputs

Unfortunately explorer_metrics.ipynb stopped working, so here we just get a unique list of the terms to cross-reference instead.



In [66]:
import os
from collections import defaultdict

import argilla as rg
from dotenv import load_dotenv, find_dotenv
import spacy

In [38]:
load_dotenv(find_dotenv(), override=True)

nlp = spacy.blank("en")

concepts = ["climate-related-hazards", "renewables"]

DATASET_NAMES = [f"explorer-quality-testing-{concept}" for concept in concepts]

rg.init(workspace="gst", api_key=os.environ["ARGILLA_API_KEY"])

rg_datasets = dict()
all_records = list()

for concept, _dataset in zip(concepts, DATASET_NAMES):
    rg_datasets[concept] = rg.load(_dataset, query="status:Validated")
    all_records.extend(rg_datasets[concept]._records)

rg_dataset = rg.DatasetForTokenClassification(all_records)
len(rg_dataset)

374

In [65]:
def get_annotation_text(rg_dataset) -> dict:
    """
    Get unique annotations (not case-sensitive) for each label in the dataset.

    Dataset is keyed by label.
    """
    results = defaultdict(list)

    for record in rg_dataset:
        record_text = record.text
        for annotation in record.annotation:
            label = annotation[0]
            start, end = annotation[1], annotation[2]
            text = record_text[start:end]
            results[label].append(text.lower().rstrip(".,;:-?"))

    results = {k: sorted(list(set(v))) for k, v in results.items()}

    return results


get_annotation_text(rg_datasets["climate-related-hazards"])["climate-related-hazards"]

["'extremes'",
 '(storms',
 'a heat wave',
 'acidification',
 'and sea ice loss',
 'associated coastal hazards',
 'associated wildfires',
 'breakdown of physical infrastructure',
 'changes to rainfall',
 'climate change induced biodegradation',
 'climate extreme',
 'climate extreme events',
 'climate extremes',
 'climate hazards',
 'climate risks are growing',
 'climate- related hazards',
 'climate-related hazards',
 'climate-related illnesses',
 'coastal and marine hazards',
 'coastal climate adaptation',
 'coastal flooding',
 'coinciding heat and drought)',
 'compound climate hazards',
 'conflict',
 'critical service systems',
 'cumulative effects of multiple human stressors in marine systems',
 'cyclones',
 'cyclones)',
 'degradation of air quality',
 'detrimental to mental health',
 'direct (primary pollutants) or indirect (secondary pollutants)',
 'displacement',
 'disrupted infrastructures',
 'drought',
 'droughts',
 'ecosystem-based disaster risk reduction',
 'emerg. manag. nat.